### Introduction

This notebook will be for environment setup and for producing the baseline result using ResNet-50. Run the **Prerequisities** section once to get the VizWiz dataset. Under **Prerequisities** there will be two sections for installing the *timm* library (contains pre-trained models). One section will be dedicated just for a local installation if you are running this on your laptop and another for a Google CoLab installation so we don't have to keep on installing the library everytime.

### Mount Google Drive

In [1]:
# Show button and code
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Prerequisites

#### Download the dataset

In this stage, we want to download all images available in VizWiz dataset. Downloading images and annotations may take a while.

In [46]:
!mkdir -p dataset/images predictions
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip

--2024-04-14 15:18:42--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298421598 (11G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  10.52G  15.4MB/s    in 12m 2s  

2024-04-14 15:30:45 (14.9 MB/s) - ‘train.zip’ saved [11298421598/11298421598]

--2024-04-14 15:30:45--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip
Reusing existing connection to vizwiz.cs.colorado.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 3488913457 (3.2G) [application/zip]
Saving to: ‘val.zip’

val.zip             100%[===================>]   3.25G  14.9MB/s    in 3m 49s  

2024-04-14 15:34:35 (14.5 MB/s) - ‘val.zip’ saved [3488913457/3488913457]

--2024-04-14 15:34:35--  https://vizwiz.cs.colorado.edu/VizWi

In [47]:
!unzip -q -o train.zip -d dataset/images
!unzip -q -o val.zip -d dataset/images
!unzip -q -o test.zip -d dataset/images

In [ ]:
!rm train.zip val.zip test.zip

#### Install timm (local)

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.5 MB/s eta 0:00:00


#### Install timm (colab)

In [3]:
!pip3 install virtualenv
!virtualenv "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env"
!chmod 755 "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env/bin/activate"; pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 51.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 24640ms
  creator CPython3Posix(dest=/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.1.0, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

### Get predictions

#### Import libraries (local)

In [4]:
import os
import argparse
import json
from datetime import datetime

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms

import timm

#### Import Libraries (CoLab)

In [50]:
import os
import argparse
import json
from datetime import datetime

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms

# changes working directory to cmsc_472 shared proj file
os.chdir("/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/")

import sys
# add the path of the virtual environmentsite-packages to colab system path
sys.path.append("virtual_env/lib/python3.10/site-packages")
import timm

#### Set variables


In [95]:
ann_path = 'dataset/annotations.json'
images_path = 'dataset/images'
prediction_path = 'predictions/'
model_name = 'resnet50'
batch_size = 64

#### Load annotation file

In [96]:
annotations = json.load(open(ann_path))
indices_in_1k = [d['id'] for d in annotations['categories']]

#### Set device

In [97]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Create dataset class and dataloader

In [98]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
    ])

class VizWizClassification(Dataset):
    def __init__(self, annotations, transform=None):
        self.images = []
        for img in annotations['images']:
          if 'train' in img:
            self.images.append(images_path + '/train/' + str(img))

          if 'val' in img:
            self.images.append(images_path + '/val/' + str(img))

          if 'test' in img:
            self.images.append(images_path + '/test/' + str(img))

        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.images[idx].split("/")[3]

dataset = VizWizClassification(annotations,test_transform)
vizwiz_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False)

#### Load the model

In [29]:
model = timm.create_model(model_name, pretrained=True).to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

#### Get predictions

In [99]:
results = {}
with torch.no_grad():
    for images, images_path in vizwiz_loader:
        images = images.to(device)
        outputs = model(images)[:,indices_in_1k]
        pred = list(outputs.data.max(1)[1].cpu())
        for i in range(len(pred)):
                results[images_path[i]] = indices_in_1k[pred[i]]

### Save the prediction file for EvalAI server

In [100]:
file_path = os.path.join(prediction_path, datetime.now().strftime("prediction-%m-%d-%Y-%H:%M:%S.json"))
with open(file_path, 'w') as outfile:
    json.dump(results, outfile)

Now you can upload this file on EvalAI server.